# Working with Corporate Bond Data in Python

<p>
The first portion of this post will focus on the opportunity for research in the corporate bond market. The second half utilizes Python to filter and analyze a corporate bond dataset. 

<p>
Harvey Campbell has written eloquently about the multiple comparison problem in asset pricing research - particularly as it pertains to the stock market. Put simply, as more researchers look for trading strategies or factors in a single dataset, it becomes more difficult to determine whether the patterns they find in the data were found due to chance or if they are really there. This problem plagues the equity market where thousands of researchers have poured over the CRSP database. 
<p>
So researchers are left with two options: generate really great back test results in equities or find new markets on which fewer researchers focus. The corporate bond market seems like a natural alternative. This $7tn market has seen massive growth as companies issued bonds under the recent low interest rate regime. In addition, there are fewer systematic strategies that have been discovered. 
<p>
Another advancement in the corporate credit market was the introduction of TRACE in 2002. TRACE came when the NASD mandated that market participants begin reporting transaction data so that it could be publicly disseminated. For the first time, researchers have real transaction data on which to test theories. 
<p>
Market participants in the corporate credit market know that the TRACE database is filled with many mistakes. This was not realized in academia until 2013 when Jan Dick-Neilsen published the second of two papers outlining how to filter mistakes from the TRACE database (See Dick-Nielsen *How to Clean Enhanced Trace Data*). Before that, research was potentially biased because Dick-Neilsen suggests a minimum of 6.5 percent of trades are filtered by his methodology. In addition, in 2012 Peter Feldhutter showed that small trades can bias analysis of transaction data from TRACE. Lastly, many bonds in the TRACE database are not corporate bonds. Combining TRACE with the FISD database allows for the separation of corporate bonds from other asset prices in the dataset. 
<p>
Below, I will implement the 3 filters (Dick-Neilsen, Feldhutter, FISD) in Python on a small group of corporate bonds and calculate the yield-to-maturity for each trade. Implementing this on the entire TRACE database (which I have done) provides a relatively clean dataset for research on systematic strategies in credit. 

# Import Relevant Packages

In [ ]:
import pandas as pd
from pandas import *
import pylab as plt
from pylab import *
import numpy as np
import csv as csv
import warnings
warnings.filterwarnings('ignore')
import scipy as scipy
from scipy import *
import matplotlib
%matplotlib inline
import statsmodels.api as sm
from statsmodels import *
import os

def side_by_side(*objs, **kwds):
    from pandas.core.common import adjoin
    space = kwds.get('space', 4)
    reprs = [repr(obj).split('\n') for obj in objs]
    print adjoin(space, *reprs)


# Importing FISD


FISD provides issue level data like type of security, offering date, offering price and coupon. This CSV comes from WRDS. It includes 350,000 assets. I show the types of information included in the dataset below. 

In [2]:
fisd_main =read_csv('FISD Bond Data.csv')
print 'Issues in FISD', len(fisd_main)
print fisd_main.columns

Issues in FISD 350063
Index([u'PROSPECTUS_ISSUER_NAME', u'ISSUER_CUSIP', u'ISSUE_CUSIP', u'MATURITY',
       u'COUPON_TYPE', u'CONVERTIBLE', u'ASSET_BACKED', u'CANADIAN', u'OID',
       u'FOREIGN_CURRENCY', u'OFFERING_AMT', u'OFFERING_DATE',
       u'OFFERING_PRICE', u'DELIVERY_DATE', u'PRINCIPAL_AMT', u'COVENANTS',
       u'DEFAULTED', u'ANNOUNCED_CALL', u'ACTIVE_ISSUE', u'BOND_TYPE',
       u'PERPETUAL', u'EXCHANGEABLE', u'PREFERRED_SECURITY', u'COMPLETE_CUSIP',
       u'FIX_FREQUENCY', u'RESET_DATE', u'RESET_DATE_ORIG',
       u'FIRST_INTEREST_DATE', u'INTEREST_FREQUENCY', u'COUPON',
       u'COUPON_CHANGE_INDICATOR', u'LAST_INTEREST_DATE', u'CUSIP_NAME',
       u'INDUSTRY_GROUP', u'INDUSTRY_CODE', u'PARENT_ID', u'NAICS_CODE',
       u'SIC_CODE'],
      dtype='object')


I take a subset of the data. There is nothing special about the bonds chosen other than they represent a relatively diverse set of corporate bonds. Then I show the head of the dataset for context. 

In [3]:
print len(fisd_main)
cusip_list = ['17453BAF8','17453BAJ0','873168AM0','873168AP3','873168AN8','873168AQ1','451663AA6',
              '816752AA7','451663AC2','800907AL1','800907AN7','882330AA1','882330AC7','458204AD6',
              '458204AF1','184502AZ5','882330AF0','882330AG8','184502BC5','184502BB7','184502BE1',
              '97314XAE4','780097AW1','97315LAA7','458204AH7','458204AJ3','69545QAA7','184502BF8',
              '695459AF4','89255MAA4','184502BG6','18451QAF5','18451QAE8','18451QAH1','18451QAG3',
              '18451QAK4','18451QAJ7']
fisd_main = fisd_main[fisd_main['COMPLETE_CUSIP'].isin(cusip_list)]
print len(cusip_list)
print len(fisd_main)

350063
37
37


In [4]:
fisd_main.head()

,PROSPECTUS_ISSUER_NAME,ISSUER_CUSIP,ISSUE_CUSIP,MATURITY,COUPON_TYPE,CONVERTIBLE,ASSET_BACKED,CANADIAN,OID,FOREIGN_CURRENCY,...,INTEREST_FREQUENCY,COUPON,COUPON_CHANGE_INDICATOR,LAST_INTEREST_DATE,CUSIP_NAME,INDUSTRY_GROUP,INDUSTRY_CODE,PARENT_ID,NAICS_CODE,SIC_CODE
102984,CITIZENS COMMUNICATIONS CO,17453B,AF8,2031/08/15,F,N,N,N,N,N,...,2,9.00,N,20310215,CITIZENS COMMUNICATIONS CO,1,11,920,51711,4813
113682,CITIZENS COMMUNICATIONS CO,17453B,AJ0,2031/08/15,F,N,N,N,N,N,...,2,9.00,N,20310215,CITIZENS COMMUNICATIONS CO,1,11,920,51711,4813
169787,TXU CORP,873168,AM0,2024/11/15,F,N,N,N,N,N,...,2,6.50,N,20240515,TXU CORP,3,30,49869,221122,4911
169788,TXU CORP,873168,AP3,2034/11/15,F,N,N,N,N,N,...,2,6.55,N,20340515,TXU CORP,3,30,49869,221122,4911
181745,TXU CORP,873168,AN8,2024/11/15,F,N,N,N,N,N,...,2,6.50,N,20240515,TXU CORP,3,30,49869,221122,4911


Next I filter based on the FISD universe. Since the bonds chosen above were all corporate bonds, only 3 of 37 will be filtered. In filtering the entire TRACE universe, you can utilize roughly 20 fields in the FISD dataset to separate undesirable bonds. 

In [5]:
print 'Issues in FISD', len(fisd_main)
fisd_main = fisd_main[(fisd_main['BOND_TYPE'] =='CDEB') | (fisd_main['BOND_TYPE'] =='CPIK') |
                      (fisd_main['BOND_TYPE'] =='CZ') | (fisd_main['BOND_TYPE'] =='CS')]

fisd_main = fisd_main[fisd_main['FOREIGN_CURRENCY'] =='N']
print len(fisd_main)

fisd_main['CUSIP_ID'] = fisd_main['ISSUER_CUSIP'] + fisd_main['ISSUE_CUSIP']
good_cusips = DataFrame(fisd_main['CUSIP_ID'])
good_cusips['IDENTIFIER'] = 1

print len(good_cusips)
print 'Issues in FISD', len(fisd_main)

Issues in FISD 37
37
37
Issues in FISD 37


# Importing the TRACE database

Here I import the relevant columns from the TRACE dataset. It represents 91mm trades and 90m unique cusips. 

In [6]:
trace =read_csv('TRACE ALL.csv',
                usecols = ['CUSIP_ID', 'TRD_EXCTN_DT','TRD_EXCTN_TM','MSG_SEQ_NB',
                'TRC_ST','ASCII_RPTD_VOL_TX','RPTD_PR','ASOF_CD','ORIG_MSG_SEQ_NB'])
trace.head()

,CUSIP_ID,TRD_EXCTN_DT,TRD_EXCTN_TM,MSG_SEQ_NB,TRC_ST,ASCII_RPTD_VOL_TX,RPTD_PR,ASOF_CD,ORIG_MSG_SEQ_NB
0,G7603ZAC5,2011/04/26,20:25:05,8555,G,1000000,100.125,A,NaN
1,05530RAB4,2012/01/24,15:09:34,30364,G,4000000,70.250,A,NaN
2,05530RAB4,2012/01/27,10:55:24,30365,G,42000,75.750,A,NaN
3,77586TAA4,2012/02/01,8:14:06,6200,G,500000,100.150,A,NaN
4,77586TAA4,2012/02/01,12:06:00,13476,H,200000,100.100,A,19490


TRACE COLUMN KEY (for details see the *TRACE User Guide* and the *TRACE Variable Guide*)
1. CUSIP_ID = Bond CUSIP or unique identifier
2. TRD_EXCTN_DT = Trade Date
3. TRD_EXCTN_TM = Trade Time
4. MSG_SEQ_NB = Identifier assigned to each trade. Not necessarily unique. 
5. TRC_ST = First Good/Bad Trade Identifier. Identifies mistakes fixed on mistaken trade's execution date. 
6. ASCII_RPTD_VOL_TX = Trade Volume. Max of 5MM for IG bonds and 1MM for HY bonds.
7. RPTD_PR = Trading Price
8. ASOF_CD = Second Good/Bad Trade Identifier. Identifies trade errors fixed after the mistaken trade's execution date.  
9. ORIG_MSG_SEQ_NB = MSG_SEQ_NB of the mistaken trade from the same date. Paired with TRC_ST.

In [7]:
print 'unique trace cusips  ',len(trace['CUSIP_ID'].unique())
print 'total trades         ',len(trace)
print 'unique fisd len      ',len(good_cusips),'(same as above)'

unique trace cusips   90839
total trades          91555441
unique fisd len       37 (same as above)


# Applying the FISD filter to TRACE

Inner merge the TRACE database with the good CUSIPs from the FISD database and we will be left with the bonds that exist in both datasets. Only 15 overlap because a large portion of the bonds above were 144a before those were included in TRACE. 144a issues have recently been added. 

There are roughly 117m trades between the 15 bonds. The last line of code generates the head of the dataset for context. 

In [8]:
trace = merge(trace,good_cusips,on=['CUSIP_ID'],how = 'inner')
trace = trace[notnull(trace['RPTD_PR'])]

In [9]:
print 'Unique CUSIPs in overlapping TRACE/FISD datasets',len(trace['CUSIP_ID'].unique())
print 'Unique Trades',len(trace)

trace.head()

Unique CUSIPs in overlapping TRACE/FISD datasets 15
Unique Trades 117262


,CUSIP_ID,TRD_EXCTN_DT,TRD_EXCTN_TM,MSG_SEQ_NB,TRC_ST,ASCII_RPTD_VOL_TX,RPTD_PR,ASOF_CD,ORIG_MSG_SEQ_NB,IDENTIFIER
0,184502BB7,2012/03/15,14:03:27,29339,G,1MM+,77.75,NaN,NaN,1
1,184502BB7,2012/03/15,15:13:02,37180,G,1MM+,78.00,NaN,NaN,1
2,184502BB7,2012/03/15,15:56:16,43642,G,1MM+,78.25,NaN,NaN,1
3,184502BB7,2012/03/16,11:46:51,15642,G,1000000,78.50,NaN,NaN,1
4,184502BB7,2012/03/19,13:08:32,22147,G,1000000,78.75,NaN,NaN,1


# Cleaning up TRACE datatypes

I clean up a few datatype issues in the TRACE dataset. 

In [10]:
trace.dtypes

CUSIP_ID              object
TRD_EXCTN_DT          object
TRD_EXCTN_TM          object
MSG_SEQ_NB             int64
TRC_ST                object
ASCII_RPTD_VOL_TX     object
RPTD_PR              float64
ASOF_CD               object
ORIG_MSG_SEQ_NB      float64
IDENTIFIER             int64
dtype: object

In [11]:
#changing date to datetime
trace['TRD_EXCTN_DT'] = trace.TRD_EXCTN_DT.str[:4] + '-' + trace.TRD_EXCTN_DT.str[5:7] + '-' + trace.TRD_EXCTN_DT.str[8:10]
trace['TRD_EXCTN_DT'] = to_datetime(trace['TRD_EXCTN_DT'])
print 'Dataset runs from', trace['TRD_EXCTN_DT'].min(), 'to', trace['TRD_EXCTN_DT'].max()

Dataset runs from 2004-09-14 00:00:00 to 2014-06-30 00:00:00


In [12]:
#changing trade size format so we can operate on sizes as floats
trace.ASCII_RPTD_VOL_TX[trace.ASCII_RPTD_VOL_TX =='1MM+' ] =  1000005
trace.ASCII_RPTD_VOL_TX[trace.ASCII_RPTD_VOL_TX =='5MM+' ] =  5000005
trace['ASCII_RPTD_VOL_TX'] = trace['ASCII_RPTD_VOL_TX'].astype(float)
len(trace), trace.ASOF_CD.unique(), trace.TRC_ST.unique()

(117262,
 array([nan, 'A', 'R', 'D', 'X'], dtype=object),
 array(['G', 'I', 'H', 'T', 'C', 'W'], dtype=object))

In [13]:
trace.dtypes

CUSIP_ID                     object
TRD_EXCTN_DT         datetime64[ns]
TRD_EXCTN_TM                 object
MSG_SEQ_NB                    int64
TRC_ST                       object
ASCII_RPTD_VOL_TX           float64
RPTD_PR                     float64
ASOF_CD                      object
ORIG_MSG_SEQ_NB             float64
IDENTIFIER                    int64
dtype: object

# Applying Dick-Nielsen - Part 1

The first part of the filter deletes trades that were canceled on the same day as the original transaciton. Same day cancels are identified by the TRC_ST column. 

The logic is:

1. If TRC_ST = H or C then delete it
  1. Also delete the trade from that day whose MSG_SEQ_NB equals the deleted H or C trade's ORIG_MSG_SEQ_NB
2. IF TRC_ST = I or W, then delete the I or W trade. 
  1. I or W means that the original trade was updated to reflect the error so deleting this fixes the problem. 

In [14]:
trace_len_pre_filter = len(trace)
print 'pre filter length', trace_len_pre_filter

#delete I/W
trace = trace[(trace['TRC_ST'] != 'I')&(trace['TRC_ST'] != 'W')]
print 'post I/W delete length', trace_len_pre_filter

#create dataframe of H/C trades
trace_same_day_cancel = trace[(trace['TRC_ST'] == 'H') | (trace['TRC_ST'] == 'C')]
trace_same_day_cancel = trace_same_day_cancel[['CUSIP_ID','TRD_EXCTN_DT','ORIG_MSG_SEQ_NB']]
trace_same_day_cancel['CANCEL_TRD'] = 1
trace_same_day_cancel = trace_same_day_cancel.rename(columns={'ORIG_MSG_SEQ_NB':'MSG_SEQ_NB'})

trace = merge(trace,trace_same_day_cancel,on=['CUSIP_ID','TRD_EXCTN_DT','MSG_SEQ_NB'],how = 'outer')
trace = trace[(trace['TRC_ST'] != 'H') & (trace['TRC_ST'] != 'C')]
trace = trace[notnull(trace['TRC_ST'])]
trace = trace[(trace['CANCEL_TRD'] != 1)]
trace = trace[(notnull(trace['TRD_EXCTN_DT']))]
trace.drop(['TRC_ST', 'CANCEL_TRD', 'ORIG_MSG_SEQ_NB'], axis=1,inplace = True)
#len trace should decline by> trace_same_day_cancel
print 'final length', len(trace)

pre filter length 117262
post I/W delete length 117262
final length 110858


# Applying Dick-Neilsen - Part 2

The second part of the filter deletes trades that were canceled on day different from the original transaction. Different day cancels are identified by the ASOF_CD column. 

The logic is:

1. If ASOF_CD = X then delete
  1. Canceled trade
2. If ASOF_CD = R, then 
  1. Delete R trade 
  2. Delete prior day trade with same CUSIP/price/size

In [15]:
#filter part 2 - different days
print len(trace)
trace = trace[(trace['ASOF_CD'] != 'X')]

trace_diff_day_cancel = trace[(trace['ASOF_CD'] == 'R')]
trace = trace[(trace['ASOF_CD'] != 'R')]

trace_diff_day_cancel = trace_diff_day_cancel[['CUSIP_ID','RPTD_PR','ASCII_RPTD_VOL_TX']]
trace_diff_day_cancel['CANCEL_TRD'] = 1

trace = merge(trace,trace_diff_day_cancel,on=['CUSIP_ID','RPTD_PR','ASCII_RPTD_VOL_TX'],how = 'outer')
trace = trace[(trace['ASOF_CD'] != 'R')]
trace = trace[(trace['CANCEL_TRD'] != 1)]
trace = trace[(notnull(trace['TRD_EXCTN_TM']))]
trace.drop(['ASOF_CD', 'CANCEL_TRD'], axis=1,inplace = True)
print len(trace)

110858
104640


Dick-Nielsen proposes a third portion of his filter. It's effectively various miscellaneous issues I find superfluous. I do not show it in this post. 

In [16]:
trace.head()

,CUSIP_ID,TRD_EXCTN_DT,TRD_EXCTN_TM,MSG_SEQ_NB,ASCII_RPTD_VOL_TX,RPTD_PR,IDENTIFIER
0,184502BB7,2012-03-15,14:03:27,29339,1000005,77.75,1
1,184502BB7,2012-03-22,9:06:46,2348,1000005,77.75,1
2,184502BB7,2012-10-12,12:15:25,17676,1000005,77.75,1
3,184502BB7,2012-10-23,16:34:20,48394,1000005,77.75,1
4,184502BB7,2012-10-25,10:41:10,9515,1000005,77.75,1


# Calculating Yield

Next I will use the Newton-Raphson method to calculate yield-to-maturity for each bond. Yield is the rate of return on a bond assuming that it receives all of its coupon payments and is paid off at some date in the future. It’s important because it allows investors to compare bonds with different prices and coupons. The process is as follows:
1. Merge relevant columns from FISD. 
2. Define functions to calculate bond price and yield.
3. Calculate Yield for each trade in the dataset. 

<p>
The error checking is very important when performing this on the entire TRACE dataset. 

In [17]:
print len(trace)
fisd_for_yield = fisd_main[['CUSIP_ID','MATURITY','COUPON','INTEREST_FREQUENCY','PRINCIPAL_AMT']]
fisd_for_yield['MATURITY'] = to_datetime(fisd_for_yield['MATURITY'])
trace = trace.merge(fisd_for_yield,on='CUSIP_ID',how='inner')
trace = trace[notnull(trace['RPTD_PR'])]
trace['Days_to_Maturity'] = (trace['MATURITY'] - trace['TRD_EXCTN_DT']).astype('timedelta64[D]')
trace['N_MATURITY'] = (trace['Days_to_Maturity'] / 365) * 2
trace = trace[trace['Days_to_Maturity'] > 0]
print len(trace)
trace.head()

104640
104640


,CUSIP_ID,TRD_EXCTN_DT,TRD_EXCTN_TM,MSG_SEQ_NB,ASCII_RPTD_VOL_TX,RPTD_PR,IDENTIFIER,MATURITY,COUPON,INTEREST_FREQUENCY,PRINCIPAL_AMT,Days_to_Maturity,N_MATURITY
0,184502BB7,2012-03-15,14:03:27,29339,1000005,77.75,1,2016-08-01,10.75,2,1000,1600,8.767123
1,184502BB7,2012-03-22,9:06:46,2348,1000005,77.75,1,2016-08-01,10.75,2,1000,1593,8.728767
2,184502BB7,2012-10-12,12:15:25,17676,1000005,77.75,1,2016-08-01,10.75,2,1000,1389,7.610959
3,184502BB7,2012-10-23,16:34:20,48394,1000005,77.75,1,2016-08-01,10.75,2,1000,1378,7.550685
4,184502BB7,2012-10-25,10:41:10,9515,1000005,77.75,1,2016-08-01,10.75,2,1000,1376,7.539726


In [18]:
def Px(Rate,Mkt_Price,Face,Freq,N,C):
    return Mkt_Price - (Face * ( 1 + Rate / Freq ) ** ( - N ) + ( C / Rate ) * ( 1 - (1 + ( Rate / Freq )) ** -N ) )

def YieldCalc(guess,Mkt_Price,Face,Freq,N,C):
    x = scipy.optimize.newton(Px, guess,args = (Mkt_Price,Face,Freq,N,C), tol=.0000001, maxiter=100)
    return x

In [19]:
yld = {}
for i in trace.index:
    try:
        yld.update({i:YieldCalc(trace['COUPON'].loc[i]/(trace['PRINCIPAL_AMT'].loc[i]/10),trace['RPTD_PR'].loc[i],
                                trace['PRINCIPAL_AMT'].loc[i]/10, trace['INTEREST_FREQUENCY'].loc[i],
                                trace['N_MATURITY'].loc[i], trace['COUPON'].loc[i])})
    except(RuntimeError):
        pass
    else:
        pass
trace['YTM']=Series(yld)
trace = trace.sort(columns = ['CUSIP_ID','TRD_EXCTN_DT','TRD_EXCTN_TM'])
trace.head()

,CUSIP_ID,TRD_EXCTN_DT,TRD_EXCTN_TM,MSG_SEQ_NB,ASCII_RPTD_VOL_TX,RPTD_PR,IDENTIFIER,MATURITY,COUPON,INTEREST_FREQUENCY,PRINCIPAL_AMT,Days_to_Maturity,N_MATURITY,YTM
24011,17453BAJ0,2004-10-01,10:41:38,3002,16000,104,1,2031-08-15,9,2,1000,9814,53.775342,0.086156
24015,17453BAJ0,2004-10-01,10:42:57,3043,25000,104,1,2031-08-15,9,2,1000,9814,53.775342,0.086156
24012,17453BAJ0,2004-10-01,10:45:12,3126,16000,104,1,2031-08-15,9,2,1000,9814,53.775342,0.086156
24045,17453BAJ0,2004-10-01,10:48:47,3237,7000,104,1,2031-08-15,9,2,1000,9814,53.775342,0.086156
24046,17453BAJ0,2004-10-01,10:50:58,3315,10000,104,1,2031-08-15,9,2,1000,9814,53.775342,0.086156


Finally, cut all trades less than 100m per with Feldhutter 2012. 

In [20]:
trace = trace[trace['ASCII_RPTD_VOL_TX'] > 99999]

# Conclusion

<p>
This post illustrates several important aspects to working with the TRACE database. And it only begins here. Before using it to analyze returns, researchers must account for callability in yields, called bonds, bonds with negative maturity, bonds with negative yields, ratings migration and a far more robust FISD filter then the one used in the simplified universe presented here. This is why most researchers are not willing to delve into TRACE data and part of the reason why systematic strategies have been so much slower to develop in credit relative to equities.